In [1]:
from anytree import Node, RenderTree, PreOrderIter
import json
from pymongo import MongoClient
import glob
import json

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [3]:
COLLECTION_NAME = 'data_false_1'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[1379138530841034752,
 1323042521971347459,
 48334468,
 1354901334558961666,
 1354822816533778433,
 726638956255698944,
 490900516,
 2379332846,
 781869059805745152,
 1260584083165454337,
 517864708,
 1969870681,
 1198819943396986880,
 1047480789628342273,
 1309151822087630849,
 1360015102645305345,
 46430723,
 738598461554102274,
 1333975668330176513,
 2338823611,
 33084616,
 787072567,
 1324186048407240706,
 824763009880952832,
 1379246843536769027,
 1366493160931790850,
 710221581989982208,
 110054643,
 1170447582272708614,
 791415826230439936,
 1286703077853192192,
 720800661500203009,
 1308807467934134274,
 337337452,
 30146450,
 1170724962061168645,
 367478647,
 1203175936478392320,
 734007202277515264,
 1358513673988415495,
 1346565114162458625,
 1300840377436561409,
 21438469,
 15688554,
 38868846,
 229981371,
 1031686679805849600,
 1339950183161978882,
 75419753,
 1043472273611591680,
 397385749,
 1212493121432387584,
 1314261789450829827,
 811982988925960192,
 197143029,
 974

In [4]:
# get time difference of parent tweet and child tweet
from datetime import datetime
def difference_datetime(child_datetime, parent_datetime):
    first_datetime = datetime.strftime(datetime.strptime(child_datetime,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    first_datetime = datetime.strptime(first_datetime, '%Y-%m-%d %H:%M:%S')
    later_datetime = datetime.strftime(datetime.strptime(parent_datetime,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    later_datetime = datetime.strptime(later_datetime, '%Y-%m-%d %H:%M:%S')
    return first_datetime - later_datetime

In [5]:
def add_depth_nodes(root, depth_n, tweet_user_ids):
    # Add depth n users
    #get the length of the node
    a = [node.path for node in PreOrderIter(root)]
#     print(a)
    names = []
    for i in range(len(a)):
        depth_of_node = len(a[i])
        if depth_of_node == depth_n:
            names.append(a[i][depth_of_node-1])
    print(names[:5])
          
    # Add depth_n nodes to the cascade
    for child in names:
        followers = db['followers'].find({'user_id': child.name})
        follower = list(followers)
        if not len(follower) == 0:
            child_followers = follower[0]['followers'] 
            child_potential_user_ids = list(set(child_followers) & set(tweet_user_ids))
            # add depth 2 users to the tree and tweet created time
            for user_id in child_potential_user_ids:
                tweets = db[COLLECTION_NAME].find({'user.id':{'$in': child_potential_user_ids}})
                for tweet in tweets:
                    if tweet['user']['id'] == user_id:
                        difference = difference_datetime(tweet['created_at'], child.created_at).total_seconds()
                        if difference >= 0:
                            globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = child)

    for pre, fill, node in RenderTree(root): 
        print("%s user_id: %s, diff_s: %s" % (pre, node.name, node.difference_seconds))
    return root

In [6]:
# remove redundant nodes
# if there are redundant user_ids keep only the most loweres diff_s
# find duplicates
import collections
import anytree
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter



def remove_duplicates(root) :
    #get the length of the node
    a = [node.path for node in PreOrderIter(root)]
#     print(a)
    names = []
    for i in range(len(a)):
        depth_of_node = len(a[i])
        names.append(a[i][depth_of_node-1].name)
    duplicates = [item for item, count in collections.Counter(names).items() if count > 1]
    print(len(duplicates))
    
    # remove duplicates    
    all_nodes_tobe_remove = []
    for duplicate in duplicates:
    #     print(duplicate)
        nodes = anytree.search.findall_by_attr(root, duplicate)
    #     print(nodes)
        node_created_at = set()
        node_different_seconds = []
        # if not crated_at is equal
            # keep the olderst node
        # else
            # keep the closest node to the parent

        # get closest node to the parent
        difference_seconds_sorted = sorted(nodes, key=lambda x: x.difference_seconds)

        for node in nodes:
            node_created_at.add(node.created_at)
            node_different_seconds.append(node.difference_seconds)

        if not len(node_created_at) ==  1:
            print('keep the olderst node', difference_seconds_sorted[0])
            all_nodes_tobe_remove.extend(difference_seconds_sorted[1:])
        else:
    #         print('keep the closest node to the parent: ', min(node_different_seconds), difference_seconds_sorted[0])
            all_nodes_tobe_remove.extend(difference_seconds_sorted[1:])
    print(len(all_nodes_tobe_remove))
    for node in all_nodes_tobe_remove:
        node.parent = None

In [18]:
def multiple_root_users(root_tweet):
    for root_user in root_tweet:
        tweets = db[COLLECTION_NAME].find({})
        followers = db['followers'].find({})
        for tweet in tweets:
            try:
                user_id = root_user
                created_at = tweet['retweeted_status']['created_at']
            except KeyError as e:
                continue    
#         print('User id:',user_id, created_at)
        root = Node(name = user_id, created_at = created_at, difference_seconds =  0)

#         print(root.name)

        follower = db['followers'].find_one({'user_id' : root.name})
        try:
            print(follower['user_id'], len(follower['followers']))
        except:
            print('None, DB does not have the user: ', root.name)
            continue

In [19]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    len(cascade_depth_one_user_ids)

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({})
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{1392673860877787139, 1392687066551513092, 1392656033496006656, 1392690962699276288, 1392703323669549061, 1392671188703334403, 1392794061237522434, 1393258351811731460, 1392933711717470220, 1394130273474908163, 1393051382735785988, 1394040157200797697, 1392770229848129538, 1392762830202474501, 1392805190760288260, 1392859348972548098, 1392819233441058816, 1392685631592042503, 1392917935363137538}
0
0


KeyboardInterrupt: 

In [11]:
for i in range(2, 10):
    root = add_depth_nodes(root, i, tweet_user_ids)
    remove_duplicates(root)

NameError: name 'root' is not defined